## Load the data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bangalore-accident-data/bangalore-cas-alerts.csv


In [2]:
df = pd.read_csv("/kaggle/input/bangalore-accident-data/bangalore-cas-alerts.csv")

In [3]:
df.head()

,deviceCode_deviceCode,deviceCode_location_latitude,deviceCode_location_longitude,deviceCode_location_wardName,deviceCode_pyld_alarmType,deviceCode_pyld_speed,deviceCode_time_recordedTime_$date
0,864504031502210,12.984595,77.744087,Kadugodi,PCW,32,2018-02-01T01:48:59.000Z
1,864504031502210,12.984595,77.744087,Kadugodi,PCW,32,2018-02-01T01:48:59.000Z
2,864504031502210,12.987233,77.741119,Garudachar Playa,FCW,41,2018-02-01T01:50:00.000Z
3,864504031502210,12.987233,77.741119,Garudachar Playa,FCW,41,2018-02-01T01:50:00.000Z
4,864504031502210,12.987503,77.740051,Hudi,Overspeed,37,2018-02-01T01:50:11.000Z


In [4]:
df.columns.values

array(['deviceCode_deviceCode', 'deviceCode_location_latitude',
       'deviceCode_location_longitude', 'deviceCode_location_wardName',
       'deviceCode_pyld_alarmType', 'deviceCode_pyld_speed',
       'deviceCode_time_recordedTime_$date'], dtype=object)

## Removing unwanted columns and duplicate rows

In [5]:
df.drop_duplicates(inplace=True)

df = df[["deviceCode_location_latitude", "deviceCode_location_longitude"]]

df.columns = ['latitude', 'longitude']

In [6]:
df.head()

,latitude,longitude
0,12.984595,77.744087
2,12.987233,77.741119
4,12.987503,77.740051
6,12.987523,77.736702
8,12.988210,77.731369


In [7]:
len(df)

152276

## Business logic: Find the number of danger spots around a given location

In [14]:
import geopandas as gpd



def find_number_danger_spots(given_lat, given_lon, distance):
    
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(
            df["longitude"],
            df["latitude"],
        ),
        crs={"init":"EPSG:4326"},
    )

    
    filtered_df = pd.DataFrame({
        "latitude" : [given_lat],
        "longitude" : [given_lon]
    })
    filtered_gdf = gpd.GeoDataFrame(
        filtered_df,
        geometry=gpd.points_from_xy(
            filtered_df["longitude"],
            filtered_df["latitude"],
        ),
        crs={"init":"EPSG:4326"},
    )
    
    gdf_proj = gdf.to_crs({"init": "EPSG:3857"})
    filtered_gdf_proj = filtered_gdf.to_crs({"init": "EPSG:3857"})
    
    x = filtered_gdf_proj.buffer(distance).unary_union
    
    neighbours = gdf_proj["geometry"].intersection(x)

    # print all the nearby points
    print(gdf[~neighbours.is_empty])

In [20]:
find_number_danger_spots(12.8434768,77.663232, 10000)

/opt/conda/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/conda/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/conda/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of 

         latitude  longitude                   geometry
2752    12.928168  77.683128  POINT (77.68313 12.92817)
2754    12.928765  77.683067  POINT (77.68307 12.92877)
2756    12.928360  77.682121  POINT (77.68212 12.92836)
2758    12.927923  77.681236  POINT (77.68124 12.92792)
2760    12.926970  77.679115  POINT (77.67912 12.92697)
...           ...        ...                        ...
207363  12.907265  77.695457  POINT (77.69546 12.90726)
207368  12.906918  77.696632  POINT (77.69663 12.90692)
207375  12.903808  77.703506  POINT (77.70351 12.90381)
207379  12.904768  77.704582  POINT (77.70458 12.90477)
207406  12.912436  77.713974  POINT (77.71397 12.91244)

[8230 rows x 3 columns]


## Save the processed data

In [21]:
df.to_csv("bangalore_danger_spots.csv", index = False)